In [1]:
from utilities.grover_state_preparation import *
from utilities.auxiliaries import *
from config import *
     
import os
import numpy as np
import pandas as pd
from scipy.stats import beta, chisquare, entropy, wasserstein_distance
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from qiskit import transpile
from qiskit.visualization import circuit_drawer
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

In [2]:
alpha = 2
beta_ = 2
a = 0
b = 1     

p_i_set = beta.pdf(np.linspace(a, b, 2**m), alpha, beta_) # changed len(2**m-1) to len(2**m)
p_i_set /= p_i_set.sum()

In [3]:
results = pd.DataFrame(columns=["ID configuration", "trainable angles", "best loss", "Earth Mover's Distance"])

n_angles = len(get_grover_angles(p_i_set, m))

for k in range(1, n_angles + 1):  
    for idx_thetas_to_optimize in combinations(range(n_angles), k):
        idx_thetas_to_optimize = list(idx_thetas_to_optimize)

        thetas_to_optimize = np.array(generate_parameters(len(idx_thetas_to_optimize), k=2))
        thetas = get_grover_angles(p_i_set, m)

        print(f"\nTesting configuration: {idx_thetas_to_optimize}")  

        best_loss = float('inf')
        best_thetas = None

        for run in range(runs):
            result = minimize(objective_function, thetas_to_optimize,
                              args=(idx_thetas_to_optimize, thetas, p_i_set, shots),
                              method=optimizer_type, options={"disp": False, "maxiter": max_iterations})
            
            current_loss = result.fun
            thetas_to_optimize = result.x
            
            if current_loss < best_loss:
                best_loss = current_loss
                best_thetas = np.array(thetas.copy())
                best_thetas[idx_thetas_to_optimize] = thetas_to_optimize 
        
        final_qc  = state_expansion(m, best_thetas)
        t_qc = transpile(final_qc, backend=backend)
        job = backend.run(t_qc, shots=shots)
        counts = job.result().get_counts(final_qc)
        samples = np.array([counts.get(state, 0) for state in all_states], dtype=float)
        samples /= samples.sum()

        
        #epsilon = 1e-10  
        #best_loss = max(best_loss, epsilon) 
        emd = wasserstein_distance(samples, p_i_set)

        
        print(f"Configuration {idx_thetas_to_optimize} -> Best Loss: {best_loss:.10f}, EMD: {emd:.10f}")


        

        new_row = pd.DataFrame([{
            "ID configuration": len(results) + 1,
            "trainable angles": str(idx_thetas_to_optimize),
            "best loss": best_loss,
            "Earth Mover's Distance": emd
        }])
        results = pd.concat([results, new_row], ignore_index=True)

saving_folder = r"C:\Users\marco\Desktop\Università\quantum\Generative Models\code"
excel_filename = os.path.join(saving_folder, "testing_m3.xlsx")
results.to_excel(excel_filename, index=False)
print(f"\nResults saved at: '{excel_filename}'")


Testing configuration: [0]
Configuration [0] -> Best Loss: 0.0622944774, EMD: 0.0080566406

Testing configuration: [1]
Configuration [1] -> Best Loss: 0.0423006572, EMD: 0.0102539062

Testing configuration: [2]
Configuration [2] -> Best Loss: 0.0484662071, EMD: 0.0073242187

Testing configuration: [3]
Configuration [3] -> Best Loss: 0.0280544278, EMD: 0.0091727121

Testing configuration: [4]
Configuration [4] -> Best Loss: 0.0383453459, EMD: 0.0093122210

Testing configuration: [5]
Configuration [5] -> Best Loss: 0.0453525560, EMD: 0.0076032366

Testing configuration: [6]
Configuration [6] -> Best Loss: 0.0384925858, EMD: 0.0083356585

Testing configuration: [0, 1]
Configuration [0, 1] -> Best Loss: 0.0269806040, EMD: 0.0070452009

Testing configuration: [0, 2]
Configuration [0, 2] -> Best Loss: 0.0239362706, EMD: 0.0052664621

Testing configuration: [0, 3]
Configuration [0, 3] -> Best Loss: 0.0358731969, EMD: 0.0061383929

Testing configuration: [0, 4]
Configuration [0, 4] -> Best Lo